In [30]:
# 필요한 라이브러리 설치
!pip install networkx matplotlib
!pip install networkx pyvis
!pip install torch torch-geometric networkx pyvis

In [41]:
# 라이브러리 임포트
import json
import torch
import networkx as nx
from pyvis.network import Network
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from IPython.display import display, HTML

# JSON 데이터 불러오기
with open('gnn_enhanced_700_dict.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# NetworkX 그래프 생성
G = nx.DiGraph()
node_mapping = {}  # 노드 ID와 인덱스 매핑

for i, node in enumerate(data['nodes']):
    G.add_node(i, label=node['word'])
    node_mapping[node['id']] = i  # 노드 매핑 저장

for edge in data['edges']:
    source = node_mapping[edge['source']]
    target = node_mapping[edge['target']]
    G.add_edge(source, target, title=edge['relation_description'])  # 엣지에 설명 추가

# PyTorch Geometric 데이터 변환
edge_index = torch.tensor(list(G.edges), dtype=torch.long).t().contiguous()
x = torch.eye(len(G.nodes))  # 임베딩을 위해 각 노드를 단위 행렬로 초기화

data = Data(x=x, edge_index=edge_index)

# GNN 모델 정의
class GNNModel(torch.nn.Module):
    def __init__(self):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(data.num_node_features, 16)
        self.conv2 = GCNConv(16, 2)  # 2차원 임베딩으로 변환

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

# 모델 학습
model = GNNModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    loss = (out[data.edge_index[0]] - out[data.edge_index[1]]).pow(2).sum()
    loss.backward()
    optimizer.step()

# PyVis 시각화를 위한 위치 지정
out = out.detach().numpy()
net = Network(notebook=True, directed=True)
for i, node in enumerate(G.nodes()):
    x, y = out[i]
    net.add_node(i, label=G.nodes[node]['label'], x=x * 100, y=y * 100)

for edge in G.edges(data=True):
    net.add_edge(edge[0], edge[1], title=edge[2]['title'])  # 엣지에 설명 추가

# PyVis 옵션 설정 및 출력
net.set_options("""
var options = {
  "nodes": {
    "font": {
      "size": 16,
      "face": "arial"
    }
  },
  "edges": {
    "arrows": {
      "to": {
        "enabled": true,
        "scaleFactor": 0.5
      }
    },
    "smooth": false
  },
  "physics": {
    "enabled": true,
    "barnesHut": {
      "gravitationalConstant": -2000,
      "springLength": 100
    }
  }
}
""")

net.show("gnn_network_with_embedding_and_tooltips.html")
display(HTML("gnn_network_with_embedding_and_tooltips.html"))


gnn_network_with_embedding_and_tooltips.html


In [43]:
 print(f"Epoch {epoch+1}/100, Loss: {loss.item():.4f}")

Epoch 100/100, Loss: 0.0000
